In [1]:
!source scgen/bin/activate
!pip install scvi-tools
!pip install git+https://github.com/theislab/scgen.git

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/scgen/bin/python3

/bin/bash: line 1: scgen/bin/activate: No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.2/380.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/

In [2]:
import sys
#if branch is stable, will install via pypi, else will install from source
branch = "stable"
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB and branch == "stable":
    !pip install --quiet scgen[tutorials]
elif IN_COLAB and branch != "stable":
    !pip install --quiet --upgrade jsonschema
    !pip install --quiet git+https://github.com/theislab/scgen@$branch#egg=scgen[tutorials]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.2 MB/s eta 0:00:00


### Loading Train Data

In [3]:
import logging
import scanpy as sc
import sklearn
import scgen
import numpy as np

In [4]:
np.random.seed(42)

In [5]:
adata = sc.read("./tests/data/train_kang.h5ad",
                backup_url='https://drive.google.com/uc?id=1r87vhoLLq6PXAYdmyyd89zG90eJOFYLk')

  0%|          | 0.00/22.4M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/usr/local/lib/python3.10/dist-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [6]:
#create test train split
train_size = int(adata.shape[0]*0.8)
test_size = adata.shape[0] - train_size


In [7]:
train_idx = np.random.choice(adata.obs_names, size = train_size, replace = False)

In [19]:
train_set = adata[adata.obs_names.isin(train_idx.tolist())].copy()
test_set = adata[~adata.obs_names.isin(train_idx.tolist())].copy()

In [8]:
train_set = adata[adata.obs_names.isin(train_idx.tolist())].copy()
test_set = adata[~adata.obs_names.isin(train_idx.tolist())].copy()

In [9]:
scgen.SCGEN.setup_anndata(train_set, batch_key="condition", labels_key="cell_type")

In [10]:
model = scgen.SCGEN(train_set)
model.save("../saved_models/model_perturbation_prediction.pt", overwrite=True)

In [11]:
model.train(
    max_epochs=100,
    batch_size=32,
    early_stopping=True,
    early_stopping_patience=25
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 26/100:  26%|██▌       | 26/100 [02:13<06:21,  5.15s/it, v_num=1, train_loss_step=65.8, train_loss_epoch=78]
Monitored metric elbo_validation did not improve in the last 25 records. Best score: 626.212. Signaling Trainer to stop.


## Prediction

In [13]:
pred, delta = model.predict(
    ctrl_key='train',
    stim_key='test',
    celltype_to_predict='CD4T'
)
pred.obs['condition'] = 'pred'

In [15]:
latent_adata.X

array([[ 0.2120054 , -0.02226838, -3.963673  , ..., -2.0464876 ,
        -2.3890455 , -0.9141015 ],
       [ 0.48850957,  3.0610886 , -2.2980022 , ...,  0.0300945 ,
        -1.5363287 , -1.3553346 ],
       [ 0.16368249, -0.6134875 , -2.3647947 , ..., -1.7214031 ,
        -2.0716903 , -1.1562297 ],
       ...,
       [ 0.25650904,  2.1763775 , -2.7843409 , ..., -2.2519078 ,
        -5.042791  , -0.85144496],
       [ 1.792957  , -1.3819628 , -0.9124117 , ..., -4.2603664 ,
        -0.80637807, -0.4689457 ],
       [ 1.9412595 , -1.5186038 , -1.9376452 , ..., -1.5313303 ,
         1.7948989 ,  1.2542162 ]], dtype=float32)

In [17]:
adata.X.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)